# Read data example

In [ ]:
import os
import typing
from tqdm import tqdm

In [ ]:
ROOT_DATA_FOLDER = "./YandexCup2024"

TRAIN_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, "YaCupTrain")
TEST_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, "YaCupTest")

In [ ]:
# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

In [ ]:
train_ids = read_testcase_ids(TRAIN_DATASET_PATH)
len(train_ids)

In [ ]:
test_ids = read_testcase_ids(TEST_DATASET_PATH)
len(test_ids)

In [ ]:
class DataFilePaths:
    def __init__(self, testcase_path: str):
        self.testcase_path = testcase_path
        
    def localization(self):
        return os.path.join(self.testcase_path, 'localization.csv')
    
    def control(self):
        return os.path.join(self.testcase_path, 'control.csv')
    
    def metadata(self):
        return os.path.join(self.testcase_path, 'metadata.json')
    
    # exists only for test_dataset
    def requested_stamps(self):
        return os.path.join(self.testcase_path, 'requested_stamps.csv')    

In [ ]:
import pandas as pd
import json

def read_localization(localization_path: str):
    return pd.read_csv(localization_path)

def read_control(control_path):
    return pd.read_csv(control_path)

def read_metadata(metadata_path: str):
    with open(metadata_path, 'r') as f:
        data = json.load(f)
    return data

def read_requested_stamps(requested_stamps_path: str):
    return pd.read_csv(requested_stamps_path)
    
def read_testcase(dataset_path: str, testcase_id: str, is_test: bool = False):
    testcase_path = os.path.join(dataset_path, str(testcase_id))
    data_file_paths = DataFilePaths(testcase_path)
    
    testcase_data = {}
    testcase_data['localization'] = read_localization(data_file_paths.localization())
    testcase_data['control'] = read_control(data_file_paths.control())
    testcase_data['metadata'] = read_metadata(data_file_paths.metadata())
    if is_test:
        testcase_data['requested_stamps'] = read_requested_stamps(data_file_paths.requested_stamps())
        
    return testcase_data

In [ ]:
def read_testcases(dataset_path: str, is_test: bool = False, testcase_ids: typing.Iterable[int] = None):
    result = {}
    if testcase_ids is None:
        testcase_ids = read_testcase_ids(dataset_path)
    
    for testcase_id in tqdm(testcase_ids):
        testcase = read_testcase(dataset_path, testcase_id, is_test=is_test)
        result[testcase_id] = testcase
    return result

In [ ]:
# may take some time

train_dataset = read_testcases(TRAIN_DATASET_PATH)
len(train_dataset)

In [ ]:
test_dataset = read_testcases(TEST_DATASET_PATH, is_test=True)
len(test_dataset)

## Dummy baseline 

### read test dataset 

In [ ]:
import numpy as np

NSECS_IN_SEC = 1000000000

def secs_to_nsecs(secs: float):
    return int(secs * NSECS_IN_SEC)

def nsecs_to_secs(nsecs: int):
    return float(nsecs) / NSECS_IN_SEC

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

### simple pose prediction logic without taking into account control states 

In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

# naive estimation of speed at last known localization pose
def dummy_estimate_last_speed(localization_poses):
    last_pose = localization_poses[-1]
    
    start_pose_idx = -1
    for i, pose in enumerate(localization_poses, start=1-len(localization_poses)):
        start_pose_idx = i
        if nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(pose['stamp_ns']) > 1.: # sec
            break
            
    start_pose = localization_poses[start_pose_idx]
    dt_sec = nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(start_pose['stamp_ns'])
    
    if dt_sec > 1e-5:
        return np.linalg.norm(last_pose['pos'][:2] - start_pose['pos'][:2]) / dt_sec
    return 5. # some default value

def dummpy_predict_pose(last_loc_pose: dict, last_speed: float, prediction_stamp: int):
    dt_sec = nsecs_to_secs(prediction_stamp) - nsecs_to_secs(last_loc_pose['stamp_ns'])
    distance = dt_sec * last_speed
    direction = yaw_direction(last_loc_pose['yaw'])
    pos_translate = direction * distance
    return {"pos": last_loc_pose['pos'] + pos_translate, 'yaw': last_loc_pose['yaw']}

In [ ]:
def predict_testcase(testcase: dict):
    loc_df = testcase['localization']
    localization_poses = localization_df_to_poses(loc_df)
    
    last_loc_pose = localization_poses[-1]
    last_speed = dummy_estimate_last_speed(localization_poses)
    
    predicted_poses = []
    for stamp in testcase['requested_stamps']['stamp_ns']:
        pose = dummpy_predict_pose(last_loc_pose, last_speed, stamp)
        predicted_poses.append(pose)
        
    predictions = {}
    predictions['stamp_ns'] = testcase['requested_stamps']['stamp_ns']
    predictions['x'] = [pose['pos'][0] for pose in predicted_poses]
    predictions['y'] = [pose['pos'][1] for pose in predicted_poses]
    predictions['yaw'] = [pose['yaw'] for pose in predicted_poses]
    return pd.DataFrame(predictions)

def predict_test_dataset(test_dataset: dict):
    predictions = {}
    for testcase_id, testcase in tqdm(test_dataset.items()): 
        predictions[testcase_id] = predict_testcase(testcase)
    return predictions

### make prediction for requested stamps 

In [ ]:
test_predictions = predict_test_dataset(test_dataset)
len(test_predictions)

### write predictions 

In [ ]:
def write_predictions(dataset_predictions: dict, prediction_file_path: str):
    prediction_list = []
    for testcase_id, prediction in tqdm(dataset_predictions.items()):
        prediction['testcase_id'] = [testcase_id] * len(prediction)
        prediction_list.append(prediction)
    predictions_df = pd.concat(prediction_list)
    predictions_df = predictions_df.reindex(columns=["testcase_id", "stamp_ns", "x", "y", "yaw"])
    print(len(predictions_df))
    predictions_df.to_csv(prediction_file_path, index=False, header=True)

In [ ]:
write_predictions(test_predictions, os.path.join(ROOT_DATA_FOLDER, "predictions.csv"))

# Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


### implementation of the metric calculation 

In [ ]:
import numpy as np
import pandas as pd

SEGMENT_LENGTH = 1.

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

def build_car_points(x_y_yaw):
    directions = np.vstack(yaw_direction(x_y_yaw[:, -1]))
    
    front_points = x_y_yaw[:, :-1] + SEGMENT_LENGTH * directions.T
    points = np.vstack([x_y_yaw[:, :-1], front_points])
    return points

def build_car_points_from_merged_df(df: pd.DataFrame):
    points_gt = df[['x_gt', 'y_gt', 'yaw_gt']].to_numpy()
    points_pred = df[['x_pred', 'y_pred', 'yaw_pred']].to_numpy()
    
    points_gt = build_car_points(points_gt)
    points_pred = build_car_points(points_pred)
    return points_gt, points_pred

def calculate_metric_testcase(df: pd.DataFrame):        
    points_gt, points_pred = build_car_points_from_merged_df(df)
    
    metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
    return metric

def calculate_metric_dataset(ground_truth_df: pd.DataFrame, prediction_df: pd.DataFrame):
    assert (len(ground_truth_df) == len(prediction_df))
    
    df = ground_truth_df.merge(prediction_df, on=['testcase_id', 'stamp_ns'], suffixes=['_gt', '_pred'])
    
    metric = df.groupby('testcase_id').apply(calculate_metric_testcase)
    return np.mean(metric)